In [80]:
import numpy as np
groundState = -1.5746

In [102]:
def build_kitaev_matrix(N, Jx=1.0, Jy=1.0, Jz=1.0):
    # Total number of sites (2 per unit cell)
    num_sites = 2 * N * N
    plack = np.zeros(N*N)
    
    for idx,i in enumerate (plack):
        if idx%4 == 0:
            plack[idx] = 1
        else:
            plack[idx] = 0
    assert np.sum(plack)%2 == 0
    # Initialize the M matrix
    M = 1j*np.zeros((num_sites, num_sites))
    zlist, xlist = generateLists(plack,N,N)
    print(xlist)
    print(zlist)
    # Loop over unit cells
    for m in range(N):
        for n in range(N):
            # Site indices for A and B in unit cell (m, n) where m is the row index and n is the collum index
            idx_A = (m * N + n) * 2  # Sublattice A
            idx_B = idx_A + 1        # Sublattice B
            
            # x-bond: A(m,n) to B(m,n) IF INSWET
            if idx_A  in xlist: 
                M[idx_A, idx_B] = -1*-2*Jx*1j
                M[idx_B, idx_A] = -1*2*Jx*1j
            else: 
                M[idx_A, idx_B] = -2*Jx*1j
                M[idx_B, idx_A] = 2*Jx*1j
            
            # y-bond: B(m,n) to A(m, n+1)
            n_next = (n + 1) % N
            idx_A_next = (m * N + n_next) * 2
            M[idx_B, idx_A_next] = 2*Jy*1j
            M[idx_A_next, idx_B] = -2*Jy*1j
            
            # z-bond: B(m,n) to A(m+1, n) IF INSWERT
            m_next = (m - 1) % N
            n_next = (n+1) %N 
            idx_A_next = (m_next * N + n_next) * 2
            if idx_A_next in zlist: 
                M[idx_B, idx_A_next] = -1*2*Jz*1j
                M[idx_A_next, idx_B] = -1*-2*Jz*1j
            else: 
                M[idx_B, idx_A_next] = 2*Jz*1j
                M[idx_A_next, idx_B] = -2*Jz*1j
                
    
    return M

In [103]:
def diagonalize_kitaev(N, Jx=1.0, Jy=1.0, Jz=1.0):
    M = build_kitaev_matrix(N, Jx, Jy, Jz)
    Mhermitian = np.conj(M.T)
    assert (np.all(np.isclose(M,Mhermitian)))
    # Diagonalize M (antisymmetric, so eigenvalues are pure imaginary or zero)
    eigenvalues = np.linalg.eigvals(M)
    # Sort eigenvalues by their real part
    eigenvalues = np.sort(eigenvalues)[::-1]
    
    # Compute ground state energy: sum of positive eigenvalues / 2
    positive_eigenvalues = eigenvalues[eigenvalues > 0]
    ground_state_energy = -0.5 * np.sum(positive_eigenvalues)
    
    return eigenvalues, ground_state_energy

In [104]:
def generateLists (plackList,nx,ny):
    PlackPairList = []
    idx1 = -1
    idx2= -1
    zlist = []
    xlist = []
    print(len(plackList))
    for idx,plack in enumerate(plackList):
        #print(idx)
        if plack == 1 and idx1 == -1 and idx2 == -1:
            idx1 = idx
        elif plack == 1 and idx1 != -1:
            PlackPairList.append((idx1,idx))
            idx1 = -1 
    #print(plackList,PlackPairList)
    for p1,p2 in PlackPairList:
        r1= p1//ny
        r2= p2//ny
        c1= p1%ny
        c2= p2%ny
        #print(r1,c1,r2,c2)
        site1 = (r1*2*nx)+2*c1 
        site2 = (r2*2*nx)+2*c2
        for col in range(0,c2-c1):
            zlist.append(site1+2*(1+col))
        if r2-r1 > 0:
            for row in range(0,r2-r1):
                xlist.append(site2-row*2*nx)
        elif r2-r1 < 0:
            for row in range(0,r1-r2):
                xlist.append(site2+((row+1)*2*nx))

    return zlist,xlist,r,c

In [113]:
# Example: N=4 lattice
N =24
eigenvalues, energy = diagonalize_kitaev(N)
print(f"Ground state energy for N={N}: {energy/N**2-groundState}")
print(f"Eigenvalues:\n{eigenvalues}")

576
[192, 144, 96, 48, 576, 528, 480, 432, 960, 912, 864, 816, 194, 146, 98, 50, 578, 530, 482, 434, 962, 914, 866, 818, 196, 148, 100, 52, 580, 532, 484, 436, 964, 916, 868, 820, 198, 150, 102, 54, 582, 534, 486, 438, 966, 918, 870, 822, 200, 152, 104, 56, 584, 536, 488, 440, 968, 920, 872, 824, 202, 154, 106, 58, 586, 538, 490, 442, 970, 922, 874, 826, 204, 156, 108, 60, 588, 540, 492, 444, 972, 924, 876, 828, 206, 158, 110, 62, 590, 542, 494, 446, 974, 926, 878, 830, 208, 160, 112, 64, 592, 544, 496, 448, 976, 928, 880, 832, 210, 162, 114, 66, 594, 546, 498, 450, 978, 930, 882, 834, 212, 164, 116, 68, 596, 548, 500, 452, 980, 932, 884, 836, 214, 166, 118, 70, 598, 550, 502, 454, 982, 934, 886, 838, 216, 168, 120, 72, 600, 552, 504, 456, 984, 936, 888, 840, 218, 170, 122, 74, 602, 554, 506, 458, 986, 938, 890, 842, 220, 172, 124, 76, 604, 556, 508, 460, 988, 940, 892, 844, 222, 174, 126, 78, 606, 558, 510, 462, 990, 942, 894, 846, 224, 176, 128, 80, 608, 560, 512, 464, 992, 944, 896,

In [1]:
print("helloworld")

helloworld
